# 課題５
## 津島 亮太

## １．多対多への拡張

In [1]:
using MyMatching

[ソースコードはこちら](https://github.com/R-Tsushima/MyMatching.jl/blob/master/src/MyMatching.jl)

とりあえず多対一、一対一のテストに通るかを試してみる。

In [2]:
Pkg.test("MyMatching")

INFO: Testing MyMatching


Test Summary:               | Pass  Total
Testing deferred acceptance |   18     18


INFO: MyMatching tests passed


通った。

多対多は[ランダムな選好表を生成する関数](https://github.com/oyamad/MatchingMarkets.jl)で例を用いてテストしてみた。

In [3]:
using MatchingMarkets

In [4]:
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

## 例1
```
In [4]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[4]:
(Array{Int64,1}[[2, 1, 3], [3, 1, 2], [1, 3, 2], [1, 3, 2], [1, 2, 3]], Array{Int64,1}[[1, 3, 2, 5, 4], [5, 1, 3, 2, 4], [3, 5, 2, 4, 1]])

In [5]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [2, 2, 3];

In [6]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[6]:
([2, 3, 1, 3, 0, 1, 2], [5, 3, 5, 1, 0, 4, 2], [1, 2, 3, 4, 6, 8], [1, 3, 5, 8])
```
受入側の枠が提案側の枠の分だけあっても選好次第でマッチングが成立しない。

## 例2
```
In [7]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[7]:
(Array{Int64,1}[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 1, 3], [2, 1, 3]], Array{Int64,1}[[2, 1, 5, 4, 3], [4, 2, 1, 5, 3], [3, 2, 4, 1, 5]])

In [8]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [2, 2, 3];

In [9]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[9]:
([1, 1, 3, 2, 3, 2, 3], [2, 1, 4, 5, 5, 4, 3], [1, 2, 3, 4, 6, 8], [1, 3, 5, 8])
```

## 例3
```
In [10]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(7, 4))

Out[10]:
(Array{Int64,1}[[4, 3], [2, 3, 4, 1], [3, 1], [1], [1, 3], [4, 3, 1], [1, 2, 3]], Array{Int64,1}[[6, 2, 5, 4, 1], [3, 6, 5, 2, 7, 1, 4], [2, 3], [5, 3]])

In [11]:
p_caps = [1, 1, 2, 1, 1, 2, 1]
r_caps = [3, 3, 1, 2];

In [12]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[12]:
([0, 2, 3, 0, 1, 1, 1, 0, 2], [6, 5, 4, 0, 7, 2, 3, 0, 0], [1, 2, 3, 5, 6, 7, 9, 10], [1, 4, 7, 8, 10])
```

いずれも望ましい結果が得られた。

## ２．受入側の戦略的行動
以下では受入側の内一人が真の選好とは異なる「ウソのリスト」を示した時、その人が得をするマッチング結果を得られるかを調べたい。

ここでは提案側と受入側が一対一のマッチングという状況を仮定する。

まず、受入側の一人一人が実際に「ウソのリスト」を示した時に得られるマッチング結果が当人にとってより望ましいものになるかを逐一調べる関数を作る。

In [8]:
function strategic_pref(m_prefs::Vector{Vector{Int}},
                        f_prefs::Vector{Vector{Int}},
                        m_matched::Vector{Int},
                        f_matched::Vector{Int},
                        p_count::Vector{Int})
#p_countは受入側が真の選好リストのマッチングの過程で提案された回数（受入、拒否どちらも含む）

    m = length(m_prefs)
    n = length(f_prefs)
    
    #「ウソのリスト」を示すことでより望ましい結果を得ることがない受入側の人の必要条件
    f_require = zeros(Int64, n)
    for i in 1:n
        if f_matched[i] == f_prefs[i][1] #真の選好リストのマッチング結果が最も望ましい
            f_require[i] = 1
        end
        if p_count[i] <= 1 #マッチングの過程を通して一度しか提案されてない人
            f_require[i] = 1
        end
    end

    for i in 1:n
        if f_require[i] == 0 #「ウソのリスト」を示すことでより望ましい結果を得る受入側の人の必要条件
            f_new_prefs = copy(f_prefs)
            for j in 1:m #jは「ウソのリスト」上で0より選好度が高い男性の人数の数
                false_prefs = collect(permutations(collect(1:1:m), j))
                for l in 1:length(false_prefs)
                    f_new_prefs[i] = false_prefs[l] #「ウソのリスト」一つ一つに対してマッチング結果がどう変わるか
                    m_new_matched, f_new_matched = my_deferred_acceptance(m_prefs, f_new_prefs)
                    if findfirst(f_prefs, f_new_matched[i]) < findfirst(f_prefs, f_matched[i])
                        print(m_prefs, f_prefs, "$i, ", f_prefs[i], f_matched[i], false_prefs[l], f_new_matched[i])
                    end
                end
            end
        end
    end
end

strategic_pref (generic function with 1 method)

false_prefsは[0, 1, 2, ... , m]を並び替えたときの0より手前の順列を全て列挙することで「ウソのリスト」全てを吟味している。ここでは、[順列を列挙するパッケージ(Combinatorics.jl)](https://github.com/JuliaMath/Combinatorics.jl/tree/master/src)を用いている。

例えばm = 4, j = 2の時、

In [24]:
collect(permutations(collect(1:1:4), 2))

Out[24]:
12-element Array{Array{Int64,1},1}:
 [1, 2]
 [1, 3]
 [1, 4]
 [2, 1]
 [2, 3]
 [2, 4]
 [3, 1]
 [3, 2]
 [3, 4]
 [4, 1]
 [4, 2]
 [4, 3]
 
によって、0より手前に二人いるときの「ウソのリスト」全てを列挙できる。

以下では[ランダムな選好表を生成する関数](https://github.com/oyamad/MatchingMarkets.jl)で生成された選好表に対して、受入側の内一人が真の選好とは異なる「ウソのリスト」を示した時、その人が得をするマッチング結果を得られるかを調べている。

In [21]:
using Combinatorics
m_num = 8 #男性の人数
f_num = 8 #女性の人数
for k in 1:100 #ランダムな選好表を何個作るか
    m_prefs, f_prefs = mat2vecs.(random_prefs(m_num, f_num))
    m_matched, f_matched, p_count = my_deferred_acceptance(m_prefs, f_prefs)
    strategic_pref(m_prefs, f_prefs, m_matched, f_matched, p_count)
end

提案側、受入側ともに8人で何度か回してみたが、数値例は得られなかった。

以下は、参考までに選好表に全員をリストしなければならない(独り身を許さない)ときに使う関数。

In [25]:
function strategic_pref2(m_prefs::Vector{Vector{Int}},
                        f_prefs::Vector{Vector{Int}},
                        m_matched::Vector{Int},
                        f_matched::Vector{Int},
                        p_count::Vector{Int})
#p_countは受入側が真の選好リストのマッチングの過程で提案された回数（受入、拒否どちらも含む）

    m = length(m_prefs)
    n = length(f_prefs)
    
    #「ウソのリスト」を示すことでより望ましい結果を得るための受入側個人の必要条件
    f_require = zeros(Int64, n)
    for i in 1:n
        if f_matched[i] == f_prefs[i][1]
            f_require[i] = 1
        end
        if p_count[i] <= 1
            f_require[i] = 1
        end
    end

    for i in 1:n
        if f_require[i] == 0
            f_new_prefs = copy(f_prefs)
            false_prefs = collect(permutations(collect(1:1:m)))
            for l in 1:length(false_prefs)
                f_new_prefs[i] = false_prefs[l] #「ウソのリスト」一つ一つに対してマッチング結果がどう変わるか
                m_new_matched, f_new_matched = my_deferred_acceptance(m_prefs, f_new_prefs)
                if findfirst(f_prefs, f_new_matched[i]) < findfirst(f_prefs, f_matched[i])
                    print(m_prefs, f_prefs, "$i, ", f_prefs[i], f_matched[i], false_prefs[l], f_new_matched)
                end
            end
        end
    end
end

strategic_pref2 (generic function with 1 method)